Random Forest classification 

In [ ]:
# Import libraries
# =============================================================================
# This loads required packages (and installs them if necessary)
l_packages <- c("raster", "rgdal", "randomForest", "glcm")
for (package in l_packages){
  if(! package %in% installed.packages()){
    install.packages(package, dependencies = TRUE)
  }
  library(package, character.only = T)
}

In [ ]:
# Set working directory - insert path to your data
setwd("data")
getwd()

In [ ]:
# Load data
# =============================================================================
# Load input hyperspectral (HS) image
img <- brick("BL_202006.tif")

In [ ]:
# Assign band names to image
names(img) <-  paste0("B", c(1:54))

In [ ]:
# Load vector training data
training <- readOGR(dsn=getwd(), layer="train_polygons") # dsn name of folder containing shp, with the name in layer
View(training)

In [ ]:
# Vizualize of image and training data for control
plot(img$B3)
plot(training, bg="transparent", add=TRUE)

In [ ]:
# Feature extraction
# =============================================================================
# Calculate GLCM textures from each spectral band
# TAKES LONG TIME!!!
out <- list()
for(band in 1:dim(img)[3]) {
  out[band] <- glcm::glcm(img[[band]], window = c(3, 3), na_opt = 'center', 
                       statistics = c("mean", "variance", "homogeneity", "contrast",
                                      "dissimilarity", "entropy", "second_moment"))
  print(paste0('Finished computing textures for band #', band, '/', dim(img)[3]))
}
textures <- stack(out)

In [ ]:
# Stack image and textures
predictors <- stack(img, textures)

In [ ]:
# Preparation of training data
# Extract raster values from training polygons
df_features <- extract(predictors, training, df=TRUE) # TAKES LONG TIME!!!

In [ ]:
# Remove rows with nodata values
training <- na.omit(df_features)
# Rename column ID to Classvalue
names(training)[names(training) == 'ID'] <- 'Classvalue'

In [ ]:
# See number of features for each class
table(training$Classvalue)

In [ ]:
# Train + Apply a RF model
# =============================================================================
# TAKES LONG TIME!!
model <- randomForest(as.factor(Classvalue) ~.,data=training,
                      ntree=100, importance=TRUE, do.trace=50) ### original model had ntree=1000

In [ ]:
# Classify image using trained model
predicted <- predict(predictors, model)

In [ ]:
# Vizualize classification
plot(predicted, main = 'Classification Random Forest')

In [ ]:
# Print information about the trained model
model

In [ ]:
# Export feature importance table
feature_importance <- model$importance
write.table(feature_importance, "BL_202006_RF_feature_importance.txt")

In [ ]:
# Save classification as a raster
writeRaster(predicted, filename="BL_202006_RF_classified.tif",overwrite=TRUE)